In [5]:
import tensorflow as tf
print(tf.config.list_physical_devices())

import gc
class FreeMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'epoch {epoch} ended, info: {logs}')
        tf.keras.backend.clear_session()
        gc.collect()

2023-10-14 15:16:10.659869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-14 15:16:12.070685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 15:16:12.075114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 15:16:12.075453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [10]:
import numpy as np
raw_data = np.load('/host/code/x_y_px_py.npy')
print(len(raw_data))

np.random.shuffle(raw_data)
data_x = [x + 1.j*y for x, y in raw_data[:,:2]]
data_y = [x + 1.j*y for x, y in raw_data[:,2:4]]


[[-20.76582146  25.30016327  -4.42938805   5.41367865]
 [-43.03113937  52.27861786  -4.42938805   5.41367865]
 [-44.05728531  53.51391602  -4.42938805   5.41367865]
 ...
 [309.90792847 157.21708679   3.89717793   2.09341168]
 [269.41137695 137.61724854   3.89717793   2.09341168]
 [228.95202637 117.90105438   3.89717793   2.09341168]]
[[ 1.47936203e+02 -6.08399277e+01  1.53575623e+00 -6.83911204e-01]
 [ 4.93015015e+02 -4.27398560e+02  6.30746984e+00 -5.20868778e+00]
 [-7.20287094e+01  2.51914349e+01 -5.44082069e+00  1.87545753e+00]
 ...
 [ 2.46447021e+02  4.35848450e+02  1.53485072e+00  2.40950513e+00]
 [ 1.02812485e+02 -4.85809669e+01  3.04041696e+00 -1.39960194e+00]
 [ 8.05252625e+02  1.72743378e+02  6.33588839e+00  1.61136818e+00]]


In [17]:
import cvnn.layers as complex_layers
from cvnn.losses import ComplexMeanSquareError
from tensorflow.keras.optimizers import SGD, Adam

model = tf.keras.models.Sequential()
model.add(complex_layers.ComplexInput(input_shape=(1,)))
model.add(complex_layers.ComplexDense(units=32, activation='pol_tanh'))
model.add(complex_layers.ComplexDense(units=32, activation='pol_sigmoid'))
model.add(complex_layers.ComplexDense(units=32, activation='pol_selu'))
model.add(complex_layers.ComplexDense(units=1, activation='linear'))
model.compile(loss=ComplexMeanSquareError(), optimizer=SGD())

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_dense (ComplexDens  (None, 32)                128       
 e)                                                              
                                                                 
 complex_dense_1 (ComplexDe  (None, 32)                2112      
 nse)                                                            
                                                                 
 complex_dense_2 (ComplexDe  (None, 32)                2112      
 nse)                                                            
                                                                 
 complex_dense_3 (ComplexDe  (None, 1)                 66        
 nse)                                                            
                                                                 
Total params: 4418 (17.26 KB)
Trainable params: 4418 (17

In [18]:
history = model.fit(np.array(data_x), np.array(data_y), shuffle=True, epochs=10, validation_split=0.2, verbose=0, callbacks=[FreeMemory()])

epoch 0 ended, info: {'loss': 6.960112571716309, 'val_loss': 7.480626583099365}
epoch 1 ended, info: {'loss': 6.855918884277344, 'val_loss': 9.164103507995605}
epoch 2 ended, info: {'loss': 6.842881679534912, 'val_loss': 8.35236644744873}
epoch 3 ended, info: {'loss': 6.837730884552002, 'val_loss': 7.061497211456299}
epoch 4 ended, info: {'loss': 6.836249351501465, 'val_loss': 12.791231155395508}
epoch 5 ended, info: {'loss': 6.833764553070068, 'val_loss': 6.9980692863464355}
epoch 6 ended, info: {'loss': 6.835825443267822, 'val_loss': 8.7809476852417}
epoch 7 ended, info: {'loss': 6.827271461486816, 'val_loss': 6.8541154861450195}
epoch 8 ended, info: {'loss': 6.826211929321289, 'val_loss': 7.2098798751831055}
epoch 9 ended, info: {'loss': 6.820016860961914, 'val_loss': 8.482335090637207}
